$ \frac{1}{4} \sum_{m<n} J_{HS}(m-n)\, \vec{\sigma}_m\cdot\vec{\sigma}_n $ where $J_{HS}(m-n) = \frac{\\pi^2}{N^2 \sin^2 ((m-n)\pi / N)}$, $N$ is number of sites. Use periodic boundary conditions

In [1]:
import numpy as np
import netket as nk
import flax.linen  as nn
from netket.operator.spin import sigmax, sigmay, sigmaz
import matplotlib.pyplot as plt
from src.hamiltonians import wrap_index

In [2]:
def haldane_shastry_model(N):
    hi = nk.hilbert.Spin(s=1/2, N=N)
    sum = 0
    for n in range(N):
        for m in range(n):
            hs = (np.pi**2) / (N**2 * (np.sin((wrap_index(N, n, m))*np.pi/N)**2))
            x = sigmax(hi, m) * sigmax(hi, n)
            y = sigmay(hi, m) * sigmay(hi, n)
            z = sigmaz(hi, m) * sigmaz(hi, n)
            sum += hs * (x+y+z)
    return sum / 4

In [4]:
nqs_en_x = []
nqs_en_y = []
analytical_en_x = []
analytical_en_y = []

In [ ]:
# N = 28

# hs = haldane_shastry_model(N)
# rbm = nk.models.RBM(alpha=1, param_dtype=complex, kernel_init=nn.initializers.normal(stddev=0.01))

# hi = nk.hilbert.Spin(s=1/2, N=N)
# sampler = nk.sampler.MetropolisLocal(hi)

# vstate = nk.vqs.MCState(sampler, model=rbm, n_samples=500)
# optimizer = nk.optimizer.Sgd(learning_rate=0.1)
# preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
# vmc = nk.driver.VMC(hs, optimizer, variational_state=vstate, preconditioner=preconditioner)

# log = nk.logging.RuntimeLog()
# print('expecting energy', -np.pi**2 / 24 * (N + 5 / N))
# vmc.run(n_iter=250 + 15 * N, out=log)
# opt_energy = vstate.expect(hs)

/Users/lzawbrito/opt/miniconda3/envs/nqs/lib/python3.12/site-packages/netket/vqs/mc/mc_state/state.py:257: UserWarning: n_samples=500 (500 per device/MPI rank) does not divide n_chains=16, increased to 512 (512 per device/MPI rank)
  self.n_samples = n_samples


expecting energy -11.587973024493309


 30%|██▉       | 200/670 [00:36<01:26,  5.43it/s, Energy=-10.092+0.008j ± 0.019 [σ²=0.186, R̂=1.0696]] 


SystemError: CPUDispatcher(<function LocalOperator._get_conn_flattened_kernel at 0x168a93240>) returned a result with an exception set

In [13]:
for N in range(27, 31):
    if not N % 2 == 0:
        continue
    
    print(f'running: N={N}')

    hs = haldane_shastry_model(N)
    rbm = nk.models.RBM(alpha=1*(N/10), param_dtype=complex, kernel_init=nn.initializers.normal(stddev=0.01))

    hi = nk.hilbert.Spin(s=1/2, N=N)
    sampler = nk.sampler.MetropolisLocal(hi)

    vstate = nk.vqs.MCState(sampler, model=rbm, n_samples=512 + 16 * N)
    optimizer = nk.optimizer.Sgd(learning_rate=0.1)
    preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
    vmc = nk.driver.VMC(hs, optimizer, variational_state=vstate, preconditioner=preconditioner)

    true_energy = -np.pi**2 / 24 * (N + 5 / N) 
    print('expecting energy', true_energy)

    log = nk.logging.RuntimeLog()
    vmc.run(n_iter=500 + 15 * N, out=log)
    opt_energy = vstate.expect(hs)

    nqs_en_x.append(N)
    analytical_en_x.append(N)
    nqs_en_y.append(opt_energy.mean.real)
    analytical_en_y.append(true_energy)

running: N=28
expecting energy -11.587973024493309


  1%|▏         | 12/920 [00:17<22:27,  1.48s/it, Energy=11.145+0.047j ± 0.030 [σ²=0.893, R̂=1.0121]]   


KeyboardInterrupt: 

In [ ]:
np.savetxt("../data/hs_outputs_1.txt", np.column_stack((nqs_en_x, nqs_en_y, analytical_en_x, analytical_en_y)))